In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
from skimage.measure import regionprops, label, find_contours
from scipy.spatial.distance import cdist

In [3]:
#値を-1から1に正規化する関数
def normalize_x(image):
    return image / 127.5 - 1


def denormalize_x(image):
    return (image + 1) * 127.5


#値を0から1正規化する関数
def normalize_y(image):
    return image / 255


#値を0から255に戻す関数
def denormalize_y(image):
    return image * 255


In [4]:
def load_Y_gray_with_gaussian_and_numeric(folder_path, thresh = None , normalize = True, g_size = None):
    image_files = []
    #image_files = os.listdir(folder_path)
    
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(int(base))
        else:
            pass
        
    image_files.sort()
    #print(image_files)
    
    img = cv2.imread(folder_path + os.sep + str(image_files[0]).zfill(4) + ".png", cv2.IMREAD_GRAYSCALE)
    
    images = np.zeros(
        (len(image_files), img.shape[0], img.shape[1], 1) ,np.float32
    )
    
    for i , image_file in enumerate(image_files):
        image = cv2.imread(
            folder_path + os.sep + str(image_file).zfill(4) + ".png" ,
            cv2.IMREAD_GRAYSCALE
        )
        #print(image.shape)
        
        # ぼかし処理
        if g_size:
            image = cv2.GaussianBlur(
                image, (g_size, g_size), 0
            )
        
        
        
        if thresh:
            ret , image = cv2.threshold(image , thresh , 255 , cv2.THRESH_BINARY)
        image = image[ : , : , np.newaxis]
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
            
    print(images.shape)
    
    return images , image_files

In [6]:
def calc_iou(im1, im2):
    intersection = np.logical_and(im1, im2)
    union = np.logical_or(im1, im2)
    iou_score = np.sum(intersection) / np.sum(union)
    if np.sum(union) == 0:
        iou_score = 0
    return iou_score

In [5]:
baseline_imgs , _ = load_Y_gray_with_gaussian_and_numeric("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/tap_dataset/cropped_0_20/baseline/", normalize=False)
tap_imgs , _ = load_Y_gray_with_gaussian_and_numeric("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/tap_dataset/cropped_0_20/tap/", normalize=False)
suga_imgs , _ = load_Y_gray_with_gaussian_and_numeric("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/tap_dataset/cropped_0_20/suga/", normalize=False)


(20, 200, 200, 1)
(20, 200, 200, 1)
(20, 200, 200, 1)


In [7]:
baseline_iou = list()

for i in range(baseline_imgs.shape[0] - 1):
    iou = calc_iou(baseline_imgs[i], baseline_imgs[i+1])
    baseline_iou.append(iou)
    
baseline_iou

C:\Users\Neurobiology\Anaconda3\envs\philow_data_manuscript\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in long_scalars
  after removing the cwd from sys.path.


[0,
 0.0,
 0.20754716981132076,
 0.0,
 0.0,
 0.2932405566600398,
 0.7854864433811802,
 0.5559120073327223,
 0.8282582216808769,
 0.7688695068768869,
 0.8688037993469873,
 0.9152357590624148,
 0.9175851703406813,
 0.9520515267175572,
 0.927828204560549,
 0.9618824531516184,
 0.9603271983640081,
 0.9465737514518002,
 0.969770935035674]

In [8]:
tap_iou = list()

for i in range(tap_imgs.shape[0] - 1):
    iou = calc_iou(tap_imgs[i], tap_imgs[i+1])
    tap_iou.append(iou)
    
tap_iou

C:\Users\Neurobiology\Anaconda3\envs\philow_data_manuscript\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in long_scalars
  after removing the cwd from sys.path.


[0,
 0.0,
 0.22119815668202766,
 0.5119617224880383,
 0.6682769726247987,
 0.6516264428121721,
 0.7593625498007968,
 0.7533013205282113,
 0.8067796610169492,
 0.8003101977510663,
 0.8150490041100221,
 0.8820412716118238,
 0.9171355498721228,
 0.9340659340659341,
 0.9350011168192986,
 0.9545842217484009,
 0.9577292219726363,
 0.9523531699727732,
 0.9638238050609185]

In [9]:
suga_iou = list()

for i in range(suga_imgs.shape[0] - 1):
    iou = calc_iou(suga_imgs[i], suga_imgs[i+1])
    suga_iou.append(iou)
    
suga_iou

[0.0,
 0.5247524752475248,
 0.3447098976109215,
 0.6540178571428571,
 0.7521222410865874,
 0.6424642464246425,
 0.7694915254237288,
 0.7763684913217623,
 0.853195164075993,
 0.6857142857142857,
 0.7584126984126984,
 0.8833942118572633,
 0.9369870267407996,
 0.9251548946716233,
 0.9029564432408035,
 0.9536093648384999,
 0.9556299706990372,
 0.9507507507507508,
 0.946917808219178]

In [11]:
print("xy")
print("baseline : ", round(np.var(baseline_iou[4:14]), 4))
print("TAP : ", round(np.var(tap_iou[4:14]), 4))
print("Manual : ", round(np.var(suga_iou[2:14]), 4))

xy
baseline :  0.0889
TAP :  0.0082
Manual :  0.0237
